In [3]:
import numpy as np
import time
import scipy

In [5]:
def maxvol1(A, I, J, U = None, V = None, maxit=7):
    row = I[np.random.randint(0, len(I))]
    if U is None:
        for it in range(1, maxit+1, 2):
            col = J[np.argmax(np.abs(A[row, I]))]
            row = I[np.argmax(np.abs(A[J, col]))]
    else:
        for it in range(1, maxit+1, 2):
            col = J[np.argmax(np.abs(A[row, J] - U[row, :] @ V[:, J]))]
            row = I[np.argmax(np.abs(A[I, col] - U[I, :] @ V[:, col]))]
    return row, col

In [34]:
def mCross(A, eps = 1e-5):
    n, m = A.shape
    I = np.arange(n)
    J = np.arange(m)
    i, j = maxvol1(A, I, J)
    U = (A[:, j] / np.abs(A[i, j])**0.5).reshape(-1, 1)
    V = (A[i, :] * np.abs(A[i, j])**0.5 / A[i, j]).reshape(1, -1)
    I = np.setdiff1d(I, [i])
    J = np.setdiff1d(J, [j])
    r = 1
    UVnorm2 = np.linalg.norm(U @ V)**2
    while True:
        i, j = maxvol1(A, I, J, U, V)
        UV_ij = U[i, :] @ V[:, j]
        if np.abs(A[i, j] - UV_ij) * np.sqrt((m-r) * (n-r)) < eps * np.sqrt(UVnorm2):
            break
        u = (A[:, j] - U @ V[:, j]) / np.sqrt(np.abs(A[i, j] - UV_ij))
        v = (A[i, :] - U[i, :] @ V) * np.sqrt(np.abs(A[i, j] - UV_ij)) / (A[i, j] - UV_ij)
        UVnorm2 = UVnorm2 + np.linalg.norm(u)**2 * np.linalg.norm(v)**2 + np.dot(U.T @ u, V @ v.T) + np.dot(V @ v.T, U.T @ u)
        U = np.concatenate((U, u.reshape(-1, 1)), axis=1)
        V = np.concatenate((V, v.reshape(1, -1)), axis=0)
        I = np.setdiff1d(I, [i])
        J = np.setdiff1d(J, [j])
        r += 1
    return (U, V, r)

In [8]:
def Hilbert(i, j):
    return 1 / (i + j + 1)

M = 8192
N = 8192

A = np.fromfunction(Hilbert, [M, N])

In [13]:
start = time.time()
u, s, v = np.linalg.svd(A, full_matrices=False)
end = time.time()
print("Classical time: ", end - start, " sec")

Classical time:  108.8213403224945  sec


In [39]:
eps = 1e-6

norm = np.sum(s)
cur_norm = norm.copy()
r = 0
while cur_norm >= eps:
    cur_norm -= s[r]
    r += 1
r, np.sum(s) - np.sum(s[: r])

(17, 6.071561928777669e-07)

In [40]:
start = time.time()
U, V, r = mCross(A, eps)
np.linalg.norm(A - U @ V), r
end = time.time()
print("Cross time: ", end - start, " sec")
print("Rank = {}, Relative approximation error = {}".format(r, np.linalg.norm(A - U @ V) / np.linalg.norm(A)))

Cross time:  0.31001734733581543  sec
Rank = 21, Relative approximation error = 1.9599914699476138e-08
